In [1]:
from tensorflow import keras

In [2]:
conv = {}
sep_conv = {}
dense = {}
max_pool = {}
avg_pool = {}

In [3]:
models = {
    "Xception": keras.applications.Xception,
    "VGG16": keras.applications.VGG16,
    "VGG19": keras.applications.VGG19,
    "ResNet50": keras.applications.ResNet50,
    "ResNet101": keras.applications.ResNet101,
    "ResNet152": keras.applications.ResNet152,
    "ResNet50V2": keras.applications.ResNet50V2,
    "ResNet101V2": keras.applications.ResNet101V2,
    "ResNet152V2": keras.applications.ResNet152V2,
    "InceptionV3": keras.applications.InceptionV3,
    "InceptionResNetV2": keras.applications.InceptionResNetV2,
    "MobileNet": keras.applications.MobileNet,
    "MobileNetV2": keras.applications.MobileNetV2,
    "DenseNet121": keras.applications.DenseNet121,
    "DenseNet169": keras.applications.DenseNet169,
    "DenseNet201": keras.applications.DenseNet201,
    "NASNetMobile": keras.applications.NASNetMobile,
    "NASNetLarge": keras.applications.NASNetLarge,
    "EfficientNetB0": keras.applications.EfficientNetB0,
    "EfficientNetB1": keras.applications.EfficientNetB1,
    "EfficientNetB2": keras.applications.EfficientNetB2,
    "EfficientNetB3": keras.applications.EfficientNetB3,
    "EfficientNetB4": keras.applications.EfficientNetB4,
    "EfficientNetB5": keras.applications.EfficientNetB5,
    "EfficientNetB6": keras.applications.EfficientNetB6,
    "EfficientNetB7": keras.applications.EfficientNetB7,
}

In [4]:
unknown = {}
for name, function in models.items():
    if "DenseNet" in name or "NASNet" in name:
        model = function(
            include_top=True,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling=None,
            classes=1000,
        )
    else:
        model = function(
            include_top=True,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling=None,
            classes=1000,
            classifier_activation="softmax",
        )
    print(model.name)
    for idx, layer in enumerate(model.layers):
        #print(idx)
        if isinstance(layer, keras.layers.Conv2D):
            conv[model.name+":"+layer.name] = {
                "filters": layer.filters,
                "kernel": layer.kernel_size,
                "strides":layer.strides,
                "padding":layer.padding,
                "dilation" : layer.dilation_rate,
                "groups": layer.groups,
                "input shape":tuple(layer.input.shape),
                "output shape":tuple(layer.output.shape),
                "output dtype": layer.dtype,
                "compute dtype": layer.compute_dtype,
                "workload" : "conv2d",
                #"other":layer.get_config()
            }
        elif isinstance(layer, keras.layers.SeparableConv2D):
            sep_conv[model.name+":"+layer.name] = {
                "filters": layer.filters,
                "depth multiplier": layer.depth_multiplier,
                "kernel": layer.kernel_size,
                "strides":layer.strides,
                "padding":layer.padding,
                "dilation" : layer.dilation_rate,
                "groups": layer.groups,
                "input shape":tuple(layer.input.shape),
                "output shape":tuple(layer.output.shape),
                "output dtype": layer.dtype,
                "compute dtype": layer.compute_dtype,
                "workload" : "sep_conv2d",
                #"other":layer.get_config()
            }
        elif isinstance(layer, keras.layers.Dense):
            dense[model.name+":"+layer.name] = {
                "units": layer.units,
                "input shape":tuple(layer.input.shape),
                "output shape":tuple(layer.output.shape),
                "output dtype": layer.dtype,
                "compute dtype": layer.compute_dtype,
                "workload" : "dense",
                #"other":layer.get_config()
            }
        elif isinstance(layer, keras.layers.MaxPooling2D):
            max_pool[model.name+":"+layer.name] = {
                "pool_size":layer.pool_size,
                "strides":layer.strides,
                "padding":layer.padding,
                "input shape":tuple(layer.input.shape),
                "output shape":tuple(layer.output.shape),
                "output dtype": layer.dtype,
                "compute dtype": layer.compute_dtype,
                "workload" : "max_pool2d",
                #"other":layer.get_config()
            }
        elif isinstance(layer, keras.layers.AveragePooling2D):
            avg_pool[model.name+":"+layer.name] = {
                "pool_size":layer.pool_size,
                "strides":layer.strides,
                "padding":layer.padding,
                "input shape":tuple(layer.input.shape),
                "output shape":tuple(layer.output.shape),
                "output dtype": layer.dtype,
                "compute dtype": layer.compute_dtype,
                "workload" : "avg_pool2d",
                #"other":layer.get_config()
            }
        else:
            #print("unknown layer:",idx, layer.name, type(layer))
            unknown[model.name+":"+layer.name] = type(layer)

2022-02-14 11:47:48.213247: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 11:47:48.213644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 11:47:48.244549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 11:47:48.244936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 11:47:48.245284: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

xception
vgg16
vgg19
resnet50
resnet101
resnet152
resnet50v2
resnet101v2
resnet152v2
inception_v3
inception_resnet_v2
mobilenet_1.00_224
mobilenetv2_1.00_224
densenet121
densenet169
densenet201
NASNet
NASNet
efficientnetb0
efficientnetb1
efficientnetb2
efficientnetb3
efficientnetb4
efficientnetb5
efficientnetb6
efficientnetb7


In [5]:
len(conv)
filter_sizes = set()
kernel_sizes = set()
stride_sizes = set()
padding_configs = set()
dilation_sizes = set()
group_sizes = set()
for name, config in conv.items():
    #print(name)
    filter_sizes.add(config["filters"])
    kernel_sizes.add(config["kernel"][0])
    stride_sizes.add(config["strides"][0])
    dilation_sizes.add(config["dilation"][0])
    group_sizes.add(config["groups"])
    padding_configs.add(config["padding"])

In [5]:
import json
json_txt = json.dumps(conv)
with open("conv_layer_config.json", "w") as file:
    file.write(json_txt)

In [6]:
import json
json_txt = json.dumps(sep_conv)
with open("sep_conv_layer_config.json", "w") as file:
    file.write(json_txt)

In [7]:
import json
json_txt = json.dumps(dense)
with open("dense_layer_config.json", "w") as file:
    file.write(json_txt)

In [8]:
import json
json_txt = json.dumps(max_pool)
with open("max_pool_layer_config.json", "w") as file:
    file.write(json_txt)

In [9]:
import json
json_txt = json.dumps(avg_pool)
with open("avg_pool_layer_config.json", "w") as file:
    file.write(json_txt)

In [8]:
dense = {}
max_pool = {}
avg_pool = {}

2627